In [12]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os

# Ensure output directory exists
output_dir = "output_charts"
os.makedirs(output_dir, exist_ok=True)

# Load data
fill_df = pd.read_csv("data/test_fill.csv")
pnl_df = pd.read_csv("data/test_pnl.csv")
order_df = pd.read_csv("data/test_order.csv")

# Calculate Final Profit
final_profit = pnl_df['Cumulative PnL'].iloc[-1]

# Calculate Cumulative Returns
initial_pnl = pnl_df['Cumulative PnL'].iloc[0]
pnl_df['Cumulative Returns'] = pnl_df['Cumulative PnL'] / initial_pnl

# Calculate Sharpe Ratio
returns = pnl_df['Cumulative PnL'].diff().dropna()
mean_returns = returns.mean()
std_dev_returns = returns.std()
risk_free_rate = 0  # Assuming 0 for simplicity
sharpe_ratio = (mean_returns - risk_free_rate) / std_dev_returns

# Calculate Maximum Drawdown
cumulative_pnl = pnl_df['Cumulative PnL']
rolling_max = cumulative_pnl.cummax()
drawdown = (cumulative_pnl - rolling_max) / rolling_max
max_drawdown = drawdown.min()

# Extract metrics for analysis
net_pnl = pnl_df["Cumulative PnL"].to_numpy()
time_pnl = pnl_df["Time"].to_numpy()
net_pnl_percent = net_pnl / initial_pnl

cumulative_returns = net_pnl_percent[-1]
pnl_std = net_pnl_percent.std()
sharpe_ratio_final = cumulative_returns / pnl_std
max_pnl, min_pnl = np.max(net_pnl), np.min(net_pnl)
max_drawdown_final = (min_pnl - max_pnl) / (initial_pnl + max_pnl)
final_val = initial_pnl + net_pnl[-1]

# Summary of Calculated Metrics
metrics_summary = {
    "Initial Value" : 10000000,
    "Final Profit": final_profit,
    "Sharpe Ratio": sharpe_ratio,
    "Maximum Drawdown": max_drawdown,
    "Cumulative Returns": cumulative_returns,
    "Max PnL": max_pnl,
    "Min PnL": min_pnl
}

# Print a summary of all metrics
print("\nSummary of Metrics:")
for key, value in metrics_summary.items():
    print(f"{key}: {value}")

# Save metrics summary as a table image
metrics_df = pd.DataFrame(metrics_summary.items(), columns=['Metric', 'Value'])
fig, ax = plt.subplots(figsize=(8, 4))  # Adjusted figure size
ax.axis('tight')
ax.axis('off')

# Add and style the table
table = ax.table(
    cellText=metrics_df.values,
    colLabels=metrics_df.columns,
    cellLoc='center',
    loc='center',
)
table.auto_set_font_size(False)
table.set_fontsize(10)  # Set smaller font size
table.scale(1.5, 1.5)  # Adjust cell size
for key, cell in table.get_celld().items():
    cell.set_edgecolor('black')
    cell.set_linewidth(0.5)
    if key[0] == 0:
        cell.set_facecolor('#D3D3D3')  # Highlight header row

plt.title('Metrics Summary', fontsize=16, weight='bold', pad=10)
plt.savefig(os.path.join(output_dir, 'metrics_summary_table.png'), bbox_inches='tight')
plt.close()

# Create the PnL chart
fig = go.Figure()

# Add the PnL line
fig.add_trace(go.Scatter(
    x=pnl_df['Time'],
    y=pnl_df['Cumulative PnL'],
    mode='lines',
    name='PnL',
    line=dict(width=2)
))

# Add titles and labels
fig.update_layout(
    title='PnL Over Time',
    xaxis_title='Timestamp',
    yaxis_title='PnL ($)',
    template='plotly_dark'
)

# Show the plot
fig.show()

# Save the chart as an image file
fig.write_image(os.path.join(output_dir, "pnl_chart.png"))

# Create the Intraday Price Movements chart
fill_df['TradeTime'] = pd.to_datetime(fill_df['TradeTime'])
fig = go.Figure()

# Add the Price Movements line
fig.add_trace(go.Scatter(
    x=fill_df['TradeTime'],
    y=fill_df['Price'],
    mode='lines',
    name='Price Movements',
    line=dict(width=2, color='green')
))

# Add titles and labels
fig.update_layout(
    title='Intraday Price Movements',
    xaxis_title='Trade Time',
    yaxis_title='Price ($)',
    template='plotly_dark'
)

# Show the plot
fig.show()

# Save the chart as an image file
fig.write_image(os.path.join(output_dir, "price_movements_chart.png"))



Summary of Metrics:
Initial Value: 10000000
Final Profit: -406478.999554
Sharpe Ratio: -0.006228589143656835
Maximum Drawdown: -15.599304018378206
Cumulative Returns: -105071.26204317166
Max PnL: 37280.722346
Min PnL: -544272.599554


/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

